# Effect of Informal Care on Labor Supply: A Macroeconomic Perspective

## 2018.3.12 ICES, Hosei University

### Daisuke Moriwaki

Cyberagent, Inc., AI Lab        
Keio University, School of Medicine
<daisuke.moriwaki@gmail.com>   github: daimoriwaki

In [1]:
using QuantEcon
using Plots
using Interpolations
using Distributions

# Motivation
* In 2017, it is reported taht there is 6 million care revievers in Japan by MLHW. The number is expected to increase as the baby boomers, or Dankai generations become "late elderly".
* It is unclear whether the Japanese LTCI system is sustanable in the sense of financially, which has been pointed out by several researchers.
* More importantly, the impact of care needs on economy itself (oppose to accountant's view) should be studied.


* It is easy to predict that "caregiver ratio" (the number of caregivers per one care receiver) explodes in the near future.
* Aside from the financial sustainability, the explosion of caregiver ratio cause several problems
    * insufficient care for elderly (insufficient availability of cheap nursing home, *tokuyou*)
    * Poverty due to care-leave for frail older family
    * *Economy itself* due to less labor supply
* Such a wide range of problem should be tackled by macroeconomic model rather than causal analysis.

# Summary of Results
* ...

# Literature

* Briefly survery several work in the labor and macro fields.



## Labor supply and informal care (causal studies)

* moriwaki (2015, mimeo) finds negative effect from informal care only for male.
* Nishimura
* 

## Care and Macro economy
* 




## Sustainability due to aging population in Japan
* Kitao




## Complex heterogeneous agent model concerned with social security programs
* Li (EER, 2018)

# Set up *"Care Economy"*

In [66]:
# various settings

r = 0.01; # interest rate
tau = 0.1; # income tax
beta = 0.9; # discount rate
chi = 0.4; # share of leisure in util
sigma = 0.28; # intertemporal substitution
rep_rate = 0.5; # replacement rate
NH_rate = 0.9; # government formal care provision rate. ratio to needed care
wage_care = 0.7; # care workers' wage rate. ratio to all industry
retirement_age = 3; # age when household can take up pension

# maximize value for middle given asset, wage and informal care
# row is new_asset x leisure
# column is wage x h_H x asset

min_asset = 0;
max_asset = 10;
min_wage = 0;
max_wage = 10;
min_h_H =0;
max_h_H = 1;
num_grid = 9; #9
min_l = 0;
max_l = 1;
asset_grid = linspace(min_asset, max_asset, num_grid);
l_grid = linspace(min_l, max_l, num_grid);

# Demography
* 3 periods: young, middle, and old
    * young: work in the market and consume goods and leisure.Save for the future 22-44
    * middle: work both in the market and home. consume goods and leisure. Save for the future. No mortality risk. 45-64
    * old: no work in both market and home. Those with bad health receive care either in public nursing home or at home. Mortality risk is present. 65+
* Consider 3 states: now (old is less than middle and young), 10 years later (old is more than middle and young), 20 years later (old is much more than young and middle)

## Household

* The model economy is populated by many households. 
    * You may imagine either single households without gender and unknown re-production technique or many households with couples and kids where income, assets, health, and time endowment is shared. 
    * More realistic modelling is listed as future study
* wage rate is exogenous to the agent. Given wage rate, labor supply is endogeneous.
* No negative saving is allowed (consider whether we can borrow without collateral).

## Care in detail
* In this economy, every middle-aged households face the risk of becoming care giver.
* In the beginning of the period, the health status of all households is assigned. 
* Then the total *care needs* is determined. 
* The government determine how much care to provide, which is sometimes insufficient. That deliniates reality in Japan (there are 360,000 people on the waiting list).
* Unmet care needs is distributed to potential care givers (*middle*). No state dependency. Imagine that you might become a informal care giver for your uncle, spouse's siblings and so on as well as your own genetic parents and spouse. Unexpectable.
* This scenario totally igonres the positive utility from care-giving. But there are lots of tragedies associated with informal care.

## Preference
* simple and general utility function. e.g. Yue (2018, EER)

In [67]:
function utility(consumption::Real, leisure::Real,chi::Real = chi, sigma::Real = sigma) 
    if c <= 0 | l <= 0
        return -1000
    else
        return ((c^(1-chi)*(l^chi))^(1-sigma))/(1 - sigma)
    end
end

utility (generic function with 3 methods)

## consumption
* consumption is deterimined by assets, income and saving
* there is a social assistence for the poor

In [68]:
function consumption(asset_new::Real, income::Real, asset::Real, r::Real)
    (1 + r)*asset + income - asset_new # allow negative consumption
end

consumption (generic function with 1 method)

## income
* income is determined by hourking hour in market and tax


In [69]:
function income(hour_home::Real, leisure::Real, pension::Real, wage::Real, tau::Real = tau)
    (1 - tau)*(wage*(1 - leisure - hour_home) + pension)
end

income (generic function with 2 methods)

## pension
* pension is taken up by old.
* the level of benefit is determined by wage history and replacement rate

In [70]:
"calculate pension 0 for before retirement age"
function calc_pension(age::Int32, wage_history::Array{Real, 1}, rep_rate::Real = rep_rate, retirement_age::Int32 = retirement_age)
    if age >= retirement_age
        return rep_rate*mean(wage_history)
    else
        return 0
    end
end

calc_pension

## wage
* wage evolves according to $w_{i,t} = \mu + \rho w_{i, t-1} + \epsilon_{t}$. 
* need to be complicated when finer horizon is embedded.
* use tauchen method to generate transition matrix

In [71]:
# wage trainsition matrix

rho = 0.9 # param for wage function
sigma = 0.1# param for wage function standard deviation for epsilon
mu = 0 # mean wage
n_std = 9 # length of grid

wage_markovchain = tauchen(num_grid, rho, sigma, mu, n_std);
wage_trans_mat = wage_markovchain.p;
wage_grid = exp(wage_markovchain.state_values);

## Informal care
* Middle-aged household is assigned care needs randomly.
* The level of care is represented as required hour for care at home.
* Once all the household draw the raw hour at home $h^H$, they are sum up as $\sum_i h^H_i$
* The total demand for care $D^H = \sum_i I (h_i \in (0,  0.1)$
* The unmet care needs $D^{H, unmet} = D^H - NH$ is distributed out to proportionaly to $h^H$
* Numerical excercise is done without empirical studies, which should be done in the future.

In [72]:
# hour at home transition matrix
# calcurate probability at midpoint between each grid 

rho_hour_home = 0 # param for wage function
sigma_hour_home = 1 # param for wage function standard deviation for epsilon
mu_hour_home = - 4 # mean wage
n_std = 4 # length of grid

hour_home_markovchain = tauchen(num_grid, rho_hour_home, sigma_hour_home, mu_hour_home, n_std);
hour_home_trans_mat = hour_home_markovchain.p;
hour_home_grid = exp(hour_home_markovchain.state_values);

# trancate at 0.9
for i in 1:num_grid
    if hour_home_grid[i] > 0.9
        hour_home_grid[i] = 0.9
    end
end


In [73]:
# determine hour at home
"clculate unmet care and determine hour at home."
function calc_hour_home(raw_hour_home::Real, health_mat_old::Array{Real, 1}, NH_rate::Real = NH_rate)
    raw_hour_home*(1 - NH_rate)*count(i ->(0<=i<0.1), health_mat)
end


calc_hour_home

## labor supply
* labor supply is endogeneous
* young and middle household optimizes labor supply as well as consumption (saving) for given time endowment
* time endowment is varying across agent due to informal care
* From the first order condition $l = c(1 - \tau)w \frac{\chi}{1 - \chi}$

In [74]:
function calc_hour_mkt(leisure::Real, hour_home::Real)
    1 - leisure - hour_home
end

calc_hour_mkt (generic function with 1 method)

## Government
* government provide insufficient formal care funded by social security premium contributed by working age, which is resemble to Japanese LTCI
* cost of formal care is determined by personal expenses of informal care worker

# Quantitative Exercise

## Solution
* The model is solved (ideally) by method of simulated moment.
* So far no calibration made
* Optimization by households is done by backward induction.

## Value function
* value function optimization $v_t = max_{c_t, l_t} (u(c_t, l_t) + \beta \mathbb{E}\left( v_{t+1}(a_{t+1}) \right))$ 

In [75]:
# value function
function value_function(age::Int, utility::Real, expected_value::Real)
    utility + expected_value
end

value_function (generic function with 1 method)

In [12]:
# caluculating value for middle
# need calculaation for average wage
function value_for_middle(a_new::Real, l::Real, a::Real, w::Real, h_H::Real, v_new::Real, tau::Real = tau, beta::Real = beta)
    y = w*(1 - tau)*(1 - h_H - l)
    #c = -100
    c = (1 + r)*a + y - a_new
    if c <= 0  || l <= 0 || 1 - h_H - l < 0
        v = -100
    else
        v = util(c, l) + beta*v_new
    end
    return v, a_new, l, a, w, h_H, y, c
end

value_for_middle (generic function with 3 methods)

In [13]:
# calculate value for each input and return maximum value for each
# warinig: no interpolation embeded!
function generate_max_values_matrix(value_function::Function, asset_grid::LinSpace = asset_grid, l_grid::LinSpace = l_grid, wage_grid::LinSpace = wage_grid, 
        h_H_grid::LinSpace = h_H_grid, num_grid::Int = num_grid)
    # initializing matrices
    value_matrix = zeros(num_grid^2, num_grid^3);
    c_mat = zeros(num_grid^2, num_grid^3);
    l_mat = zeros(num_grid^2, num_grid^3);
    y_mat = zeros(num_grid^2, num_grid^3);
    a_new_mat = ones(num_grid^2, num_grid^3);
    w_mat = zeros(num_grid^2, num_grid^3);
    a_mat = zeros(num_grid^2, num_grid^3);
    h_H_mat = zeros(num_grid^2, num_grid^3);
    for i = 1:num_grid
        for j = 1:num_grid
            for k = 1:num_grid
                for m = 1:num_grid
                    for n = 1:num_grid
                        result = value_function(asset_grid[i], l_grid[j], asset_grid[k], wage_grid[m], h_H_grid[n])
                        value_matrix[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[1]
                        a_new_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[2]
                        l_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[3]
                        a_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[4]
                        w_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[5]
                        h_H_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[6]
                        y_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[7]
                        c_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2]  = result[8]
                    end
                end
            end
        end
    end
    # get max value for each asset, wage, h_H. maximizing with respect to new asset and leisure
    #max_value = maximum(value_matrix, 1)
    max_value, max_ind = findmax(value_matrix, 1)
    # reshape value matrix to asset_grid x (w x h_H)
    # this should be fix throughout the imulation
    max_values_reshaped = reshape(max_value, num_grid, num_grid^2);
    return max_values_reshaped, max_ind, a_new_mat, l_mat, a_mat, w_mat, h_H_mat, y_mat, c_mat
end


generate_max_values_matrix (generic function with 6 methods)

In [14]:
# get policy function for a_new, h_M (Labor supply), c, 
max_values_reshaped, max_ind, a_new_mat, l_mat, a_mat, w_mat, h_H_mat, y_mat, c_mat = generate_max_values_matrix(value_for_middle);
# policy function for new asset
a_new_pol_middle = a_new_mat[max_ind]
# policy function for labor supply
l_pol_middle = l_mat[max_ind]
h_M_pol_middle = 1 - l_pol_middle - reshape(h_H_mat[1, :], 1, num_grid^3)
c_pol_middle = c_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]


LoadError: MethodError: no method matching generate_max_values_matrix(::#value_for_middle, ::LinSpace{Float64}, ::LinSpace{Float64}, ::Array{Float64,1})[0m
Closest candidates are:
  generate_max_values_matrix(::Function, ::LinSpace{T<:AbstractFloat}, ::LinSpace{T<:AbstractFloat}) at In[13]:6
  generate_max_values_matrix(::Function, ::LinSpace{T<:AbstractFloat}, ::LinSpace{T<:AbstractFloat}, [1m[31m::LinSpace{T<:AbstractFloat}[0m) at In[13]:6
  generate_max_values_matrix(::Function, ::LinSpace{T<:AbstractFloat}, ::LinSpace{T<:AbstractFloat}, [1m[31m::LinSpace{T<:AbstractFloat}[0m, [1m[31m::LinSpace{T<:AbstractFloat}[0m) at In[13]:6
  ...[0m

In [15]:
plot(reshape(c_pol_middle, 729, 1))
#plot(hcat(reshape(w_mat[1,:], 729,1), reshape(l_pol_middle, 729, 1)))

LoadError: UndefVarError: c_pol_middle not defined

In [16]:
# need to be removed after get_value_for_middle_with_uncertainty is modified.

"get index for a value between grid point"
function get_float_index(x::Real, grid::LinSpace, num_grid::Int = num_grid)
# get floating index from asset grid for given scalar asset level
    if x < grid[1]
        return 1
    end
    if x> grid[num_grid]       
        return num_grid
    end
    for i = 1:num_grid
        if x >= grid[i]
            return i + (x - grid[i])/(grid[i+1] - grid[i])
        end
    end
end

get_float_index

In [17]:
max_values_matrix = generate_max_values_matrix(value_for_middle)[1]

LoadError: MethodError: no method matching generate_max_values_matrix(::#value_for_middle, ::LinSpace{Float64}, ::LinSpace{Float64}, ::Array{Float64,1})[0m
Closest candidates are:
  generate_max_values_matrix(::Function, ::LinSpace{T<:AbstractFloat}, ::LinSpace{T<:AbstractFloat}) at In[13]:6
  generate_max_values_matrix(::Function, ::LinSpace{T<:AbstractFloat}, ::LinSpace{T<:AbstractFloat}, [1m[31m::LinSpace{T<:AbstractFloat}[0m) at In[13]:6
  generate_max_values_matrix(::Function, ::LinSpace{T<:AbstractFloat}, ::LinSpace{T<:AbstractFloat}, [1m[31m::LinSpace{T<:AbstractFloat}[0m, [1m[31m::LinSpace{T<:AbstractFloat}[0m) at In[13]:6
  ...[0m

In [18]:
############ need refactoring. interpolate accept "gridded" interpolation that replace get_float_index function!!!!!!!!!!!!!!!!!

# interpolation for a (new asset for young, asset in the last period for middle)
# get a = a_new and return value for middle with uncertainty
function get_value_for_middle_with_uncertainty(a_new::Real, asset_grid::LinSpace = asset_grid, max_values_matrix = max_values_matrix)
    v = zeros(num_grid^2, 1)
    a_new_ind = get_float_index(a_new, asset_grid) # get index of asset level in asset grid
    for i = 1:size(max_values_matrix, 2)
        itp = interpolate(max_values_matrix[:,i],BSpline(Linear()), OnGrid())
        v[i] = itp[a_new_ind]
    end
    return v
end

get_value_for_middle_with_uncertainty (generic function with 3 methods)

In [19]:
#plotly()
#plot(value_for_middle_mat[:, 30])
#plot(reshape(max_values, num_grid^3, 1))
#plot(max_values_reshaped)


## Distribution 
* assign probability distributions for wage and care need to each bin arbitrary
* wage follows lognormal, care need follow gamma distribution

In [20]:
# generate probability distribution for w and h_H
# wage is distributed according to Gauss


# calcurate probability at midpoint between each grid 
function get_Q(dist, grid::LinSpace, num_grid::Int=num_grid)
    Q = zeros(num_grid, 1)
    for i in 1:num_grid
        if i == 1
            Q[i] = cdf(dist, (grid[i] + grid[i + 1])/2)
        end
        if i == num_grid
            Q[i] = 1 - cdf(dist, (grid[i - 1] + grid[i])/2) 
        end
        if i > 1 && i < num_grid
            Q[i] = cdf(dist, (grid[i] + grid[i + 1])/2) - cdf(dist, (grid[i - 1] + grid[i])/2)
        end
    end
    return Q
end



get_Q (generic function with 2 methods)

In [21]:
w_dist = LogNormal(0.5, 0.5)
h_H_dist = Gamma(0.6, 0.1) # Gamma(0.5, 0.1)

# generate Q
Q_h_H = get_Q(h_H_dist, h_H_grid);
Q_w = get_Q(w_dist, wage_grid);
Q = kron(Q_w, Q_h_H);

LoadError: UndefVarError: h_H_grid not defined

In [22]:
# calculating value for young
function value_for_young(a_new::Real, l::Real, w::Real, Q::Array{Float64, 2}=Q, max_values_matrix::Array{Real, 2}=max_values_matrix, a::Real = 0, h_H::Real = 0, asset_grid::LinSpace = asset_grid, tau::Real = tau, beta::Real = beta)
    y = w*(1 - tau)*(1 - h_H - l)
    #c = -100
    c = (1 + r)*a + y - a_new
    if l < 0 || c < 0 || h_H + l > 1 || h_H + l < 0
        v = -10000
    else
        v = util(c, l) + beta*dot(Q, get_value_for_middle_with_uncertainty(a_new, asset_grid, max_values_matrix))
    end
return v, a_new, l, a, w, h_H, y, c
end

value_for_young (generic function with 8 methods)

In [23]:
# calculating value for young
function value_for_young(a_new::Real, l::Real,  a::Real, w::Real, h_H::Real, v_new, Q = Q, asset_grid::LinSpace = asset_grid, tau::Real = tau, beta::Real = beta)
    y = w*(1 - tau)*(1 - h_H - l)
    #c = -100
    c = (1 + r)*a + y - a_new
    if l < 0 || c < 0 || h_H + l > 1 || h_H + l < 0
        v = -100
    else
        v = util(c, l) + beta*dot(v_new, Q)
    end
return  v, a_new, l, a, w, h_H, y, c
end

value_for_young (generic function with 13 methods)

In [24]:
# caluculate value for young 


value_matrix = zeros(num_grid^2, num_grid^3);
c_mat = zeros(num_grid^2, num_grid^3);
l_mat = zeros(num_grid^2, num_grid^3);
y_mat = zeros(num_grid^2, num_grid^3);
a_new_mat = ones(num_grid^2, num_grid^3);
w_mat = zeros(num_grid^2, num_grid^3);
a_mat = zeros(num_grid^2, num_grid^3);
h_H_mat = zeros(num_grid^2, num_grid^3);
for i = 1:num_grid
    for j = 1:num_grid
        for k = 1:num_grid
            for m = 1:num_grid
                for n = 1:num_grid
                    v_new = get_value_for_middle_with_uncertainty(asset_grid[i])# value for middle with uncertainty
                    result = value_for_young(0, l_grid[j], 0, wage_grid[m], 0, v_new)
                    value_matrix[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[1]
                    a_new_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[2]
                    l_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[3]
                    a_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[4]
                    w_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[5]
                    h_H_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[6]
                    y_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[7]
                    c_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2]  = result[8]
                end
            end
        end
    end
end
# get max value for each asset, wage, h_H. maximizing with respect to new asset and leisure
max_value, max_ind = findmax(value_matrix, 1)
# reshape value matrix to asset_grid x (w x h_H)
# this should be fix throughout the imulation
max_values_reshaped = reshape(max_value, num_grid, num_grid^2);

LoadError: UndefVarError: max_values_matrix not defined

In [25]:
# get policy function for a_new, h_M (Labor supply), c, 
# policy function for new asset
a_new_pol_young = a_new_mat[max_ind]
# policy function for labor supply
l_pol_young = l_mat[max_ind]
h_M_pol_young = 1 - l_pol_young - reshape(h_H_mat[1, :], 1, num_grid^3)
c_pol_young = c_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]

LoadError: UndefVarError: max_ind not defined

In [26]:
h_M_pol_young

LoadError: UndefVarError: h_M_pol_young not defined

In [27]:
max_values_reshaped

LoadError: UndefVarError: max_values_reshaped not defined

In [28]:
?findmax


search: findmax findmax! findmin findmin!



```
findmax(itr) -> (x, index)
```

Returns the maximum element and its index. The collection must not be empty.

```jldoctest
julia> findmax([8,0.1,-9,pi])
(8.0,1)
```

```
findmax(A, region) -> (maxval, index)
```

For an array input, returns the value and index of the maximum over the given region.


In [29]:
# simulation 100 thousand H/H
# initializing matrices
num_period = 3;
num_hh = 100 # 100000
c_record = zeros(num_hh, num_period);
a_record = zeros(num_hh, num_period);
w_record = zeros(num_hh, num_period);
h_H_record = zeros(num_hh, num_period);
h_M_record = zeros(num_hh, num_period);



In [30]:
# generate wage, care needs streams for each household

w_record = rand(w_dist, num_hh, num_period);
w_record[:,3] = 0;
h_H_record = rand(h_H_dist, num_hh, num_period);
h_H_record[:, 1] = 0;
h_H_record[:, 3] = 0;


In [31]:
# return (c, a_new, h_M) for young hh
# prepare for interpolation
c_pol_young_reshape = reshape(c_pol_young, num_grid, num_grid, num_grid) # reshape for interpolate    
a_new_pol_young_reshape = reshape(a_new_pol_young, num_grid, num_grid, num_grid) # reshape for interpolate    
h_M_pol_young_reshape = reshape(h_M_pol_young, num_grid, num_grid, num_grid) # reshape for interpolate    
c_itp = interpolate((asset_grid, wage_grid, h_H_grid), c_pol_young_reshape, Gridded(Linear()));
a_itp = interpolate((asset_grid, wage_grid, h_H_grid), a_new_pol_young_reshape, Gridded(Linear()));
h_M_itp = interpolate((asset_grid, wage_grid, h_H_grid), h_M_pol_young_reshape, Gridded(Linear()));

for n = 1:num_hh
    c_record[n,1] = c_itp[a_record[n,1], w_record[n,1], h_H_record[n,1]]
    a_record[n,2] = a_itp[a_record[n,1], w_record[n,1], h_H_record[n,1]]
    h_M_record[n,1] = h_M_itp[a_record[n,1], w_record[n,1], h_H_record[n,1]]
end




LoadError: UndefVarError: c_pol_young not defined

In [32]:
# return (c, a_new, h_M) for middle

c_pol_middle_reshape = reshape(c_pol_middle, num_grid, num_grid, num_grid);
a_new_pol_middle_reshape = reshape(a_new_pol_middle, num_grid, num_grid, num_grid);
h_M_pol_middle_reshape = reshape(h_M_pol_middle, num_grid, num_grid, num_grid);
c_itp = interpolate((asset_grid, wage_grid, h_H_grid), c_pol_middle_reshape, Gridded(Linear()));
a_itp = interpolate((asset_grid, wage_grid, h_H_grid), a_new_pol_middle_reshape, Gridded(Linear()));
h_M_itp = interpolate((asset_grid, wage_grid, h_H_grid), h_M_pol_middle_reshape, Gridded(Linear()));


for n = 1:num_hh
    c_record[n,2] = c_itp[a_record[n,2], w_record[n,2], h_H_record[n,2]]
    a_record[n,3] = a_itp[a_record[n,2], w_record[n,2], h_H_record[n,2]]
    h_M_record[n,2] = h_M_itp[a_record[n,2], w_record[n,2], h_H_record[n,2]]
end

for n = 1:num_hh
    c_record[n,3] = (1+r)*a_record[n, 3] + (w_record[n,1] + w_record[n, 2])*0.5*0.5 # replacement rate for pension is 0.5 
end

LoadError: UndefVarError: c_pol_middle not defined

In [33]:
c_record

100×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮            
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [34]:
w_record

100×3 Array{Float64,2}:
 1.7172    0.74223   0.0
 0.497032  2.14812   0.0
 2.5634    1.81366   0.0
 4.21174   0.70432   0.0
 2.58298   0.87541   0.0
 2.89888   1.33878   0.0
 1.86241   1.46512   0.0
 0.826473  1.00818   0.0
 1.39806   3.04071   0.0
 1.61832   3.50944   0.0
 1.55635   0.475853  0.0
 2.27026   1.08106   0.0
 1.23573   1.38966   0.0
 ⋮                      
 2.54967   3.92804   0.0
 1.20119   1.93934   0.0
 1.94525   1.56805   0.0
 3.266     0.636711  0.0
 1.11703   1.52669   0.0
 1.43758   2.2582    0.0
 1.4532    1.11717   0.0
 0.751996  1.05857   0.0
 1.40182   0.723792  0.0
 1.2258    2.57429   0.0
 1.71014   1.43039   0.0
 2.94678   0.832542  0.0

In [35]:
h_M_record

100×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮            
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [36]:
h_H_record

100×3 Array{Float64,2}:
 0.0  3.50261e-5   0.0
 0.0  0.00888364   0.0
 0.0  0.056108     0.0
 0.0  0.244471     0.0
 0.0  0.0100593    0.0
 0.0  0.131151     0.0
 0.0  0.000168562  0.0
 0.0  0.00931944   0.0
 0.0  0.0750918    0.0
 0.0  0.0269208    0.0
 0.0  0.01561      0.0
 0.0  0.0555228    0.0
 0.0  0.0112862    0.0
 ⋮                    
 0.0  0.00961016   0.0
 0.0  0.00355482   0.0
 0.0  0.00155578   0.0
 0.0  0.013112     0.0
 0.0  0.107732     0.0
 0.0  0.0830429    0.0
 0.0  0.0169219    0.0
 0.0  0.117021     0.0
 0.0  0.0178899    0.0
 0.0  0.0026578    0.0
 0.0  0.0743193    0.0
 0.0  0.0491543    0.0

In [37]:
sum(h_M_record)

0.0

In [38]:
sum(c_record)

0.0